# Geofence: Cutdown Method
## 1.1 Standard Implementation
***
### 1.1.1 Initialization
**Note.** For using Jupyter Notebook locally (tested for 6.2.0) on Ubuntu (or other Debian distros)<sup>1</sup>,
two requirements are necessary:
- Python (Python 3.0 specfically) must be installed to local machine (should be included?)
    - To check if you have python already: `python --version`
- Necessary libraries/modules must also be installed to local machine that are not already included by Python (sys).

```
sudo apt update
sudo apt install python3-pip
sudo python3 -m pip install <module>
```


For documentation purposes, using the following commands will install all the necessary libraries to execute this program:

`sudo python3 -m pip install numpy pandas shapely`



In [ ]:
from sys import exit
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon

# Import "predicted coordinates" + altitude
MAX_ALTITUDE = 10
X = 35.73123
Y = -117.43313
H = 5

def predict(x=X, y=Y):
    return Point(x, y)

def altitude(h=H):
    return h

def cutdown():
    return 'Success!'

def createMap(regions):
    A = []
    for coords in regions:
        shape = []
        for s in coords.split():
            lat = s.split(",")
            shape.append(Point(float(lat[1]), float(lat[0])))
        poly = Polygon(shape)
        A.append(poly)
    return A

def inRed(poly_map):
    for shape in poly_map:
        if predict().within(shape): return True
    return False

def loop(poly_map):
    
    while altitude() < MAX_ALTITUDE: continue
        
    # Case #1: If after max altitude, it's at a RED ZONE, do nothing.
    while inRed(poly_map): continue
    
    # Case #2: Else, it's at a WHITE ZONE, do nothing.
    while not inRed(poly_map): continue
    
    cutdown()
            
# Import direct map KML (See #1.2)
# convert_to_csv("./Geofence21.kmz")

regions = pd.read_csv("./Geofence21.csv", sep='\t')["coordinates"]
poly_map = createMap(regions)

# function for runtime
loop(poly_map)
exit()


## 1.2 KML/KMZ to CSV Conversion
***
Convert Geofence KML/KMZ file from Google Maps using Python parsing + compression libraries.

### 1.2.1 Breakdown
Documentated explanations of conversion script, segmented by relevant categories of interest. Will eventually re-compile to a singular script to be uploaded as a PR.

Takes Keyhole Markup Language Zipped (KMZ) as input. The output returns a csv file. All core functionality from http://programmingadvent.blogspot.com/2013/06/kmzkml-file-parsing-with-python.html.

```
Parameters ::
file : {string} The string path to KMZ
output: {string} Defines type of output. Valid output (as of now) is .csv files. Defaults to .csv files.
```


#### 1.2.1.1 File Matching with Regex 
***
Once the string (location of file) is given, Python's regex library is used to determine if the string is a valid KML/KMZ file. If so, it will return a string (ext) that contains the data extension (in lowercase: kmz/kml). Otherwise, it will throw an error if the string is nonexistent, invalid, or there is some other OS compilation error.

In [ ]:
import re

file = './Geofence21.kmz'
r = re.compile(r'(?<=\.)km+[lz]?')

try:
    # return data extension (kml/kmz) from RE substring
    ext = r.search(file).group().lower()
except IOError as e:
    # error-logging for non-existent file/OS
    logging.error("I/O Error {}".format(e))

#### 1.2.1.2 Creating the Buffer
***
Using ZipFile and some numpy shorthands, the file will be converted to a KML file (if not already), and stored in a buffer for later use. It will throw errors if the extension is neither KML nor KMZ.

For more details on the implementation for KMZ to KML conversion:
- Using ZipFile, the KMZ file is opened to read-only.
- Using numpy's vectorize function + lambda for quick conditional statements, the program will sift through `namelist()` using ZipFile for the KML file.
- The vectorize function, in particular, will return a boolean-valued array that can act as an index for opening the KML file (See [Advanced Indexing with NumPy](https://www.pythonlikeyoumeanit.com/Module3_IntroducingNumpy/AdvancedIndexing.html))


In [ ]:
from zipfile import ZipFile
import numpy as np

if (ext == 'kml'):
    buffer = file

elif (ext == 'kmz'):
    kmz = ZipFile(file, 'r')
    
    # Search through array (namelist()) for matching KML file
    match = np.vectorize(lambda name:bool(r.search(name)))
    names = np.array(kmz.namelist())
    
    # points to correct index in names (bool-arr index)
    sel = match(names) 
    
    buffer = kmz.open(names[sel][0], 'r')
    
    
else:
    raise ValueError("Incorrect file format.")

#### 1.2.1.3 Parsing w/ XML's ContentHandler
***
 For more information on how the Handler class functions, see below: 

> Your ContentHandler handles the particular tags and attributes of your flavor(s) of XML. A ContentHandler object provides methods to handle various parsing events. Its owning parser calls ContentHandler methods as it parses the XML file.
The methods startDocument and endDocument are called at the start and the end of the XML file. The method characters(text) is passed character data of the XML file via the parameter text.
The ContentHandler is called at the start and end of each element. Here, name is the XML element <>, and attributes is an Attributes object.

With this implementation, there are three class functions that are used to parse through each KML (parseable as XML) element. In order to convert to a DataFrame (i.e. CSV table) using Pandas, I use a map (or dictionary) to collect the following (essential) information:

- Placemark (used to indicate a new KML element/object)
- Name (name of the polygon)
- StyleURL (CSS-styled polygon tags)
- Coordinates 
- Tesselate (determines shape or point)
- Description (additional information)

In short, the following class functions can be described as such:
1. `startElement(self, name=String, attributes)`: Initializes start of parse by detecting Placemark (first), and then the "name" tag.
2. `endElement(self, name=String)`: Parses information in element and add buffer to our map for CSV creation.
3. `characters(self, data=String)`: Add characters to buffer.

To learn more about how a KML file is formatted, and where information is stored, check out [this source](https://developers.google.com/kml/documentation/kmlreference).

In [ ]:
import xml.sax 
import random as rand

# Note: parameter required to connect owning parser with class
class Handler(xml.sax.ContentHandler):
    def __init__(self):
        self.inName = False 
        self.inPlacemark = False
        self.map = {} 
        self.buffer = ""
        self.tag = ""
        
    # call when an element starts
    def startElement(self, name, attributes):
        if name == "Placemark": # on start Placemark tag
            self.inPlacemark = True
            self.buffer = "" 
        
        if self.inPlacemark:
            if name == "name": # on start name tag
                self.inName = True 
                
            
    # call when an element ends
    def endElement(self, name):
        
        # clean whitespace from buffer
        self.buffer = self.buffer.strip()
        
        if name == "Placemark": # reset on end Placemark tag
            self.inPlacemark = False
            self.tag = ""
            
        elif name == "name" and self.inPlacemark:
            self.inName = False
            
            # create random hex
            hex = "%064x" % rand.randrange(10**80)
            self.tag = hex[:16]
            
            # create new id + name entry
            self.map[self.tag] = {}
            self.map[self.tag][name] = self.buffer

            
        elif self.inPlacemark:
            self.map[self.tag][name] = self.buffer
            
        self.buffer = "" # clear buffer
        
    # call when a character is read
    def characters(self, data):
        if self.inPlacemark: # on text within tag
            self.buffer += data 

#### 1.2.1.4 Parsing with SAX (Simple API with XML)
***
From [source](https://www.tutorialspoint.com/parsing-xml-with-sax-apis-in-python):
<br>
> SAX is a standard interface for event-driven XML parsing. Parsing XML with SAX generally requires you to create your own ContentHandler by subclassing xml.sax.ContentHandler.

By creating the class earlier, we created the necessary ContentHandler to handle items in our XML. I believe this class can definitely be further optimize to suit our needs for Geofence, but for now, the information can definitely suffice with these class functions.

In [ ]:
import xml.sax

parser = xml.sax.make_parser()
handler = Handler()

parser.setContentHandler(handler)
parser.parse(buffer)

# If working with a KMZ file, close after parsing
try:
    kmz.close()
except:
    pass


#### 1.2.1.5 DataFrame for CSV Creation
***
Converting our map into a DataFrame, it can now be analyzed and read in a compact, organized manner. Moreover, it will provide the link towards creating our CSV output file.

In [ ]:
import pandas as pd

df = pd.DataFrame(handler.map).T
names = list(map(lambda name: name.lower(), df.columns))

#### 1.2.1.6 Cleaning Up
***
Ensuring that the parsed information is acutely converted to CSV, I had to deliberately refurbish some column informations so that it can be used/optimized without hassle.

In [ ]:
# Testing purposes, remove if condition when done
if "Point" in df.columns:
    df = df.drop(columns=['Point', 'LinearRing', 'outerBoundaryIs', 'Polygon', 'LineString'])

df["coordinates"] = df["coordinates"].apply(lambda x: re.sub(r'\s+', ' ', x))
df["tessellate"] = df["tessellate"].fillna(0)
df

#### 1.2.1.7 Output to CSV
***
Ensuring that the file extension is labelled "csv", the Pandas library provides a function to readily convert the DataFrame into a CSV file and store it locally into our machine. With some necessary arguments (encoding, separation), as well as some congratulatory logging (or not...), the resulting file should be found relative to the directory of the code.

In [ ]:
# if output.lower() == 'csv':
if True:
    
    # Add CSV extension for output file
    out_ext = file[:-3] + "csv"
    
    df.to_csv(out_ext, encoding="utf-8", sep="\t")
    result = ('Converted {} to CSV '
              '& output at {}'.format(file, out_ext))
    
else:
    raise ValueError('The conversion returned no data, '
                     'check filename?')


### 1.2.2 Final Executable 

In [ ]:
import numpy as np
import pandas as pd
import random as rand
import xml.sax 
import re
from zipfile import ZipFile

class Handler(xml.sax.ContentHandler):
    def __init__(self):
        self.inName = False 
        self.inPlacemark = False
        self.map = {} 
        self.buffer = ""
        self.tag = ""
        
    # call when an element starts
    def startElement(self, name, attributes):
        if name == "Placemark":
            self.inPlacemark = True
            self.buffer = "" 
        
        if self.inPlacemark:
            if name == "name":
                self.inName = True 
                
            
    # call when an element ends
    def endElement(self, name):
        
        self.buffer = self.buffer.strip()
        
        if name == "Placemark":
            self.inPlacemark = False
            self.tag = ""
            
        elif name == "name" and self.inPlacemark:
            self.inName = False
            
            # create random hex
            hex = "%064x" % rand.randrange(10**80)
            self.tag = hex[:16]
            
            # create new id + name entry
            self.map[self.tag] = {}
            self.map[self.tag][name] = self.buffer

        elif self.inPlacemark:
            self.map[self.tag][name] = self.buffer
            
        self.buffer = ""
        
    # call when a character is read
    def characters(self, data):
        if self.inPlacemark:
            self.buffer += data 
    
def convert_to_csv(file, output="csv", shapes=True, extended=False, console=False):
    
    r = re.compile(r'(?<=\.)km+[lz]?')

    try:
        ext = r.search(file).group().lower()
    except IOError as e:
        logging.error("I/O Error {}".format(e))   

    if (ext == 'kml'):
        buffer = file

    elif (ext == 'kmz'):
        kmz = ZipFile(file, 'r')

        # Search through array (namelist()) for matching KML file
        match = np.vectorize(lambda name:bool(r.search(name)))
        names = np.array(kmz.namelist())

        sel = match(names) 
        buffer = kmz.open(names[sel][0], 'r')


    else:
        raise ValueError("Incorrect file format. Must be a KML/KMZ file.")

    parser = xml.sax.make_parser()
    handler = Handler()

    parser.setContentHandler(handler)
    parser.parse(buffer)

    try:
        kmz.close()
    except:
        pass


    df = pd.DataFrame(handler.map).T
    names = list(map(lambda name: name.lower(), df.columns))

    if (extended == False):
        df = df.drop(columns=['Point', 'LinearRing', 'outerBoundaryIs', 'Polygon', 'LineString'])

    df["coordinates"] = df["coordinates"].apply(lambda x: re.sub(r'\s+', ' ', x))
    df["tessellate"] = df["tessellate"].fillna(0)
    
    if (shapes == True):
        df = df[df["tessellate"] != 0]
        filter = df["styleUrl"].str.contains("^#line")
        df = df[~filter]
    
    if (console == True):
        display(df)
        
    if output.lower() == 'csv':
        # Add CSV extension for output file
        out_ext = file[:-3] + "csv"

        df.to_csv(out_ext, encoding="utf-8", sep="\t")
        result = ('Converted {} to CSV '
                  '& output at {}'.format(file, out_ext))
        print("Success!")

    else:
        raise ValueError('The conversion returned no data, '
                         'check filename?')



### 1.2.3 Try it!

In [ ]:
# Console-view option
convert_to_csv('./Geofence21.kmz', output='csv', shapes=True, extended=False, console=True)

In [ ]:
#Extended-data option
convert_to_csv('./Geofence21.kmz', output='csv', shapes=False, extended=True, console=True)

In [ ]:
# Default option
convert_to_csv('./Geofence21.kmz')

## 1.3 Revisions
***
### 1.3.1 Test Functions
For this, I attempted to create some time-based functions that would simulate the balloon's movement. Using Python's multithreading, the program can run multiple instances of functions simultaneously in order to check all getter functions throughout the loop() function.

**Warning.** In Jupyter Notebook, escaping out of the program before the thread is complete prevents it from re-running. Instead, let it run and remove the thread before restarting the program.

**Note.** In IJupyter, pressing I key twice may interrupt the program. or CTRL+M + i, but I usually play it safe by manually clicking "Interrupt" on the Kernel tab.


In [ ]:
import time
import numpy as np
from sys import exit
from threading import Thread, Lock


onExit = 0

class testThread (Thread):
    def __init__ (self, tID, name, delay, c = 5):
        Thread.__init__(self)
        self.tID = tID
        self.name = name
        self.c = c
        self.delay = delay
            
    def run(self):
        print("Starting: " + self.name)
        if (self.name == "Altitude"):
            print_incr(self.name, self.c, self.delay)
        elif (self.name == "Position"):
            print_line(self.name, self.c, self.delay)
        print("Exiting... " + self.name)

def print_incr(threadName, c, delay):
    cur = 0
    h = 1
    while cur <= c:
        delta = np.random.uniform(0.001, 0.01)
        h = (1.001 + delta) * h + (h * (0.23 + (42 * delta))) 
        if onExit:
            threadName.exit()
        time.sleep(delay)
        print("[%s] %s: %.5f" % (time.ctime(time.time()), threadName, h))
        cur += 1

def print_line(threadName, c, delay):
    t = 0
    while t <= c:
        delta = np.random.uniform(1, 3, size=(2,))
        x = 3.3 * t + delta[0]
        y = 5.4 * t + delta[1]
        
        if onExit:
            threadName.exit()
        time.sleep(delay)
        print("[%s] %s: (%.5f, %.5f)\n" % (time.ctime(time.time()), threadName, x, y, ))
        t += 1 
    
    

# Initialize new threads
print("Geofence: Mock Data Collection")
print("*** ")
print("Starting main thread ... ")
altitude = testThread(1, "Altitude", 1, 5)
position = testThread(2, "Position", 1, 5)
altitude.start()
position.start()

altitude.join()
position.join()
    
print("Exiting main thread ...")
print("Data collection complete, exiting...")
    


### 1.3.2 Mock Environment via Threading
**Not complete, work in progress...**

In [5]:
import time
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
from sys import exit
from threading import Thread, Lock
from queue import Queue

# auxillary functions
def createMap(regions):
    A = []
    for coords in regions:
        shape = []
        for s in coords.split():
            lat = s.split(",")
            shape.append(Point(float(lat[1]), float(lat[0])))
        poly = Polygon(shape)
        A.append(poly)
    return A

def inRed(point, poly_map):
    for shape in poly_map:
        if point.within(shape): return True
    return False

def putQueue(x, y):
    if q.empty():
        q.put((x, y))
    
    print("Current coordinates have been recorded. Proceeding ...")

def getQueue():
    return q.get()


onExit = 0
MAX_ALTITUDE = 40
q = Queue(1)

# Import direct map KML (See #1.2)
# convert_to_csv("./Geofence21.kmz")

regions = pd.read_csv("./Geofence21.csv", sep='\t')["coordinates"]
poly_map = createMap(regions)

class testThread (Thread):
    def __init__ (self, tID, name, delay, ctime = 5):
        Thread.__init__(self)
        self.tID = tID
        self.name = name
        self.delay = delay
        self.ctime = ctime
        
        # data collection
        self.h = 1
        self.x = 0
        self.y = 0
        self.point = Point(self.x, self.y)
        self.inRed = False
            
    def run(self):
        print("Starting: " + self.name)
        threadLock.acquire()
        
        log = ""
            
        if (self.name == "Altitude"):
            self.get_altitude()
            log = "Max altitude reached."
            
        elif (self.name == "Position"):
            self.x, self.y = getQueue()
            self.point = Point(self.x, self.y)
            self.inRed = inRed(self.point, poly_map)
#             self.inRed = True # for testing CASE #2 ONLY
            print("Current Position: " + str(self.x) + ", " + str(self.y))
            print("inRed? " + str(self.inRed))
            
            if (self.inRed):
                log = "White zone detected."
            else:
                log = "Red zone detected."
                
            self.get_pos()

        
        threadLock.release()
        
        print(log + "\nExiting... " + self.name)

    def get_altitude(self):
        while self.h < 50:
            delta = np.random.uniform(0.001, 0.01)
            self.h = (1.001 + delta) * self.h + (self.h * (0.23 + (42 * delta))) 
            print("[%s] %s: %.5f" % (time.ctime(time.time()), self.name, self.h))

            time.sleep(self.delay)

        putQueue(1, 1) # MUST replace with current position after MAX_ALT

    def get_pos(self):
        start = time.time()
        dt = 0
        while dt < 20:
            if self.inRed and not inRed(self.point, poly_map):
                break
                
            elif not self.inRed and inRed(self.point, poly_map):
                break
            
            dt = round(time.time() - start)
            
            # needs modification
            delta = np.random.uniform(1, 3, size=(2,))
            self.x = 1.3 * dt + delta[0]
            self.y = 2.4 * dt + delta[1]
            self.point = Point(self.x, self.y)
    
            print("[%s] %s: (%.5f, %.5f)" % (time.ctime(time.time()), self.name, self.x, self.y ))
            
            time.sleep(self.delay)
        if (dt >= 20):
            print("Exceeded time allotted. ignore all messages.")


def loop(poly_map):
    
    while altitude() < MAX_ALTITUDE: continue
        
    # Case #1: If after max altitude, it's at a RED ZONE, do nothing.
    while inRed(poly_map): continue
    
    # Case #2: Else, it's at a WHITE ZONE, do nothing.
    while not inRed(poly_map): continue
    
    cutdown()
    

threadLock = Lock()
threads = []



# Initialize new threads
print("Geofence: Mock Data Collection")
print("*** ")
print("Starting main thread ... ")
altitude = testThread(1, "Altitude", 1)
position = testThread(2, "Position", 1)

altitude.start()
position.start()

threads.append(altitude)
threads.append(position)



for t in threads:
    t.join()
    
# function for runtime
# loop(poly_map)
# exit()

print("Exiting main thread ...")
print("Data collection complete, exiting...")
            

    


Geofence: Mock Data Collection
*** 
Starting main thread ... 
Starting: Altitude
[Wed Feb 17 19:22:07 2021] Altitude: 1.41445
Starting: Position
[Wed Feb 17 19:22:08 2021] Altitude: 1.93210
[Wed Feb 17 19:22:09 2021] Altitude: 2.96310
[Wed Feb 17 19:22:10 2021] Altitude: 4.74810
[Wed Feb 17 19:22:11 2021] Altitude: 7.67275
[Wed Feb 17 19:22:12 2021] Altitude: 11.60124
[Wed Feb 17 19:22:13 2021] Altitude: 18.68841
[Wed Feb 17 19:22:14 2021] Altitude: 27.24679
[Wed Feb 17 19:22:15 2021] Altitude: 44.78522
[Wed Feb 17 19:22:16 2021] Altitude: 74.00049
Current coordinates have been recorded. Proceeding ...
Max altitude reached.
Exiting... Altitude
Current Position: 1, 1
inRed? False
[Wed Feb 17 19:22:17 2021] Position: (1.83200, 1.41242)
[Wed Feb 17 19:22:18 2021] Position: (4.10729, 4.59815)
[Wed Feb 17 19:22:19 2021] Position: (4.34909, 7.77110)
[Wed Feb 17 19:22:20 2021] Position: (6.68199, 8.97028)
[Wed Feb 17 19:22:21 2021] Position: (7.79912, 11.50789)
[Wed Feb 17 19:22:22 2021] Posi

## 1.4 Resources
1. [Threading, Queues](https://www.tutorialspoint.com/python3/python_multithreading.html)
2. [Geofence: Cutdown Reference](https://colab.research.google.com/drive/1h64AOgV0blvkRY8ph1JReOvioX0ahpOR?authuser=1)